In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F


In [2]:
# data_dir = '../../../../BottleStoodUp_atNight/Positive/'        #For the work laptop
data_dir = '../../../Images/BottleStoodUp_atNight/Positive'        #For the home laptop

In [3]:
# transform_characteristics = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor()])

transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset = datasets.ImageFolder(data_dir, transform=transform_characteristics)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

We use only the encoder part of the nertwork

In [4]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )
        
        # self.show_modules()
        self.show_weights()

    def forward(self, x):
        print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the new function")
        # print(self.modules())
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
    
    def show_weights(self):
        print("These are the weights function")
        # print(self.modules())
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == 0:
                    print("i is: ", i)
                    print("The type of the module weight is: ", type(m.weight))
                    print("The shape of the module weight is: ", m.weight.shape)
                    print("print the next module")
                    i = i +1
    def show_one_module_weights(self):
        print("Showing only one module's weight")
        # print(self.modules())
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == 0:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                    i = i +1
 

In [5]:
model_encoder = Autoencoder_latentSpace()


These are the weights function
i is:  0
The type of the module weight is:  <class 'torch.nn.parameter.Parameter'>
The shape of the module weight is:  torch.Size([110, 3, 9, 9])
print the next module


In [6]:
model_encoder.show_one_module_weights()

Showing only one module's weight
i is:  0
The weights are:  Parameter containing:
tensor([[[[ 1.4343e-02,  4.7362e-02, -3.4429e-02,  ...,  4.3460e-02,
            3.3041e-02, -3.1982e-02],
          [-4.7128e-02,  2.2652e-02, -2.2617e-02,  ..., -2.1153e-02,
            1.6970e-03,  4.9721e-02],
          [ 1.7138e-02, -3.4310e-02,  4.7574e-02,  ...,  5.0133e-02,
           -6.1535e-02, -1.1856e-03],
          ...,
          [-1.8452e-02,  4.9912e-02,  4.6239e-02,  ...,  4.4392e-04,
            5.4259e-02, -7.7481e-03],
          [-1.4064e-02, -2.9526e-03,  1.8159e-02,  ..., -4.6438e-02,
           -3.2012e-02, -1.5946e-02],
          [ 4.2826e-02, -1.1233e-02, -6.0836e-02,  ...,  4.3723e-02,
            6.2793e-02,  6.4627e-03]],

         [[-5.7896e-02, -5.6469e-02, -1.6328e-02,  ..., -5.0062e-02,
           -2.5830e-03, -8.2249e-03],
          [-2.1603e-03,  5.8914e-02,  2.2293e-02,  ...,  4.4897e-02,
           -1.2228e-02,  5.0970e-02],
          [-1.5231e-02, -2.7254e-02, -6.4076e

In [7]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )
        
        # N , 450, 1, 1
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(output_channels_layer3, output_channels_layer2, 8),  # -> 32, 225, 8, 8
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels_layer2, output_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), 
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels, input_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # N, 3, 224, 224
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
 
# Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
# Input [-1, +1] -> use nn.Tanh

In [8]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [9]:
filepath = "../../../BottlesAnomalies_TFM/models/pytorchModels/PytorchModel2"
# For loading the model 
model.load_state_dict(torch.load(filepath))
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

In [10]:
index = 0
i = 0
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        if i == index:
            print("i is: ", i)
            print("The weights are: ", m.weight)
        i = i +1

i is:  0
The weights are:  Parameter containing:
tensor([[[[-6.7870e-03, -5.9977e-03,  5.9349e-04,  ..., -3.1963e-03,
           -3.9213e-03, -3.6567e-03],
          [-6.5028e-03, -5.3889e-03, -5.1010e-03,  ..., -4.3888e-03,
           -2.7821e-02, -3.3790e-02],
          [-1.4501e-02, -2.2383e-02, -4.9993e-03,  ..., -4.7834e-03,
            5.3126e-03, -2.4254e-02],
          ...,
          [-7.1905e-03, -4.8892e-03, -4.2067e-03,  ..., -3.7277e-03,
           -3.2662e-02, -4.8444e-03],
          [ 1.3601e-02, -2.4451e-03,  1.2937e-02,  ..., -2.2916e-02,
           -3.0448e-04, -7.7624e-03],
          [-5.9227e-03,  1.0600e-02, -3.6416e-02,  ..., -5.6808e-03,
           -3.6151e-02, -1.7110e-02]],

         [[-2.6367e-02, -5.7756e-03, -2.7488e-02,  ...,  1.3288e-02,
           -4.9012e-03, -8.6235e-03],
          [-5.6408e-03, -6.7863e-03, -1.7314e-02,  ..., -1.7055e-02,
           -1.4709e-02, -4.5184e-03],
          [-3.1287e-03, -7.1006e-03, -4.9694e-03,  ..., -4.3487e-03,
         

In [11]:
layers_weights_list = []
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        weights = m.weight
        layers_weights_list.append(weights)
print("The shape of the weights list is: ", len(layers_weights_list))

The shape of the weights list is:  3


Copying the module's weight from one model to another. 

In [12]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )

    def forward(self, x):
        # print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the show modules function")
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
            
    def show_one_layer_weights(self, index):
        print("This is the one layer show function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == index:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                i = i +1
    
    def update_weights(self):
        print("updating weights function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight = layers_weights_list[i]
                i = i +1

                                        
 

In [13]:
model_encoder = Autoencoder_latentSpace()
# model.eval()

In [14]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-2.9552e-02, -3.5242e-02,  3.5795e-02,  ..., -5.7389e-02,
            2.9000e-02, -1.1331e-02],
          [-2.0752e-02,  8.1981e-03, -1.1967e-03,  ..., -6.0061e-02,
            3.3497e-02, -1.9803e-03],
          [-1.9782e-02, -6.0846e-02, -4.5608e-03,  ..., -1.8737e-02,
            5.1299e-02,  6.4082e-02],
          ...,
          [ 6.2761e-02,  9.0964e-03,  1.2138e-02,  ...,  4.3878e-02,
           -4.5969e-03,  1.1160e-02],
          [-4.4098e-03,  2.3980e-02, -1.3152e-02,  ..., -3.0375e-02,
            5.7790e-02, -2.6781e-02],
          [ 1.9386e-02, -2.9536e-02, -4.4295e-03,  ...,  3.8335e-02,
            4.4271e-02, -4.0334e-02]],

         [[ 5.4184e-02, -4.3901e-02,  1.8903e-02,  ...,  2.2028e-02,
            1.1436e-03, -4.8119e-02],
          [-4.9919e-02,  9.9418e-03,  6.1728e-02,  ...,  3.7784e-02,
            1.4287e-03,  5.8688e-02],
          [-2.6512e-02,  1.9799e-02, -2.75

In [15]:
model_encoder.update_weights()

updating weights function


In [16]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-6.7870e-03, -5.9977e-03,  5.9349e-04,  ..., -3.1963e-03,
           -3.9213e-03, -3.6567e-03],
          [-6.5028e-03, -5.3889e-03, -5.1010e-03,  ..., -4.3888e-03,
           -2.7821e-02, -3.3790e-02],
          [-1.4501e-02, -2.2383e-02, -4.9993e-03,  ..., -4.7834e-03,
            5.3126e-03, -2.4254e-02],
          ...,
          [-7.1905e-03, -4.8892e-03, -4.2067e-03,  ..., -3.7277e-03,
           -3.2662e-02, -4.8444e-03],
          [ 1.3601e-02, -2.4451e-03,  1.2937e-02,  ..., -2.2916e-02,
           -3.0448e-04, -7.7624e-03],
          [-5.9227e-03,  1.0600e-02, -3.6416e-02,  ..., -5.6808e-03,
           -3.6151e-02, -1.7110e-02]],

         [[-2.6367e-02, -5.7756e-03, -2.7488e-02,  ...,  1.3288e-02,
           -4.9012e-03, -8.6235e-03],
          [-5.6408e-03, -6.7863e-03, -1.7314e-02,  ..., -1.7055e-02,
           -1.4709e-02, -4.5184e-03],
          [-3.1287e-03, -7.1006e-03, -4.96

In [17]:
########################################################
# Calculate KDE using sklearn
from sklearn.neighbors import KernelDensity

#Get encoded output of input images = Latent space
# encoded_images = model_encoder(images)
encoded_images = []

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [18]:
print(len(encoded_images))
print(type(encoded_images))
print(type(encoded_images[0]))

179
<class 'list'>
<class 'torch.Tensor'>


See that it is 179 length because it corresponds to all the images that belong to the training dataset. 
 

Convert "encoded_images" to a np array

In [48]:
np_encoded_images = []
for i in range (len(encoded_images)):
    np_conversion = encoded_images[i].detach().numpy()
    np_encoded_images.append(np_conversion)
np_encoded_images = np.array(np_encoded_images)
print(type(np_encoded_images))

<class 'numpy.ndarray'>


In [49]:
print(np_encoded_images.shape)

(179, 450, 1, 1)


Also, see above the shape of the representation of the original images has been lowered to (1, 1) as specified in the model structure. The number 450, on the other hand, corresponds to the channels of the image; this value started at 3 and layer by layer it incremented until reaching 450.

Now, we have to flatten the data in order to apply kernel density on it.

In [27]:
model_encoder_output_shape = (450,1,1)
print(model_encoder_output_shape)

(450, 1, 1)


In [28]:
out_vector_shape = model_encoder_output_shape[0]*model_encoder_output_shape[1]*model_encoder_output_shape[2]
encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_images]


In [31]:
len(encoded_images_vector)

179

In [32]:
encoded_images_vector[0].shape

(450,)

In [58]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_images_vector)

The above function fits a kernel density estimation to the data that is provided, that is, to the "encoded_images_vector" variable. It does so using a Guassian kernel of bandwidth 0.2.

The badnwidth parameter affects on how the selected kernel will fit each sample of the given data. For example for the case in which the kernel is a Gaussian distribution, the bandwidth parameter would affect in how thin or wide is the Gaussian distribution.

At this point in the variable "kde" we have some numbers that are the result of fitting Gaussian functions to the given data points in the variable "encoded_images_vecotr". We will use the "kde" variable later for scoring with it, some given data points; the scoring will be given depending on how similar are the given data points to the ones that it had estimated.

Here below, it is shown the kde values corresponding to each encoded sample:

In [71]:
density_vals = kde.score_samples(encoded_images_vector)
print(density_vals)

[305.53733971 307.35717838 306.59277755 307.21218292 308.34709187
 307.29109145 307.7629479  307.42217212 307.58382746 307.71712255
 308.01089546 307.8728298  307.38181678 306.91408681 306.33269559
 308.01556191 308.19376503 307.57424077 307.42433853 306.62791089
 306.4966751  306.57885108 306.41965798 306.65997135 306.77791516
 306.63068956 307.29632209 307.31821178 306.41795476 306.79516573
 306.79720236 308.21580257 307.45056141 307.37195979 306.64308054
 306.49874321 307.27166825 306.73922603 305.85072122 306.53897049
 306.58100322 307.50442519 307.24766109 306.86449102 308.32323807
 306.29340382 306.55116172 306.39655809 306.28892166 306.55262643
 307.26068027 307.81019439 307.87679935 308.01688186 307.88287558
 306.61783525 308.09682984 307.84485327 308.13527638 307.87153362
 306.7652701  307.61809407 307.28747487 308.32091862 308.35797587
 307.89734813 307.91695543 308.0541587  308.03375511 307.9970379
 307.92932803 307.7780845  307.59128534 308.05939678 307.69529841
 307.940688

From here, the mean and standard deviation of these values are computed

In [72]:
average_density = np.mean(density_vals)
stdev_density = np.std(density_vals)
print("The avg of the density values is: ", average_density)
print("The stdev_density of the density values is: ", stdev_density)

The avg of the density values is:  307.54132174423586
The stdev_density of the density values is:  0.5951443460453201


As a side note, the density calculation, was computed, in the video reference (https://www.youtube.com/watch?v=q_tpFGHiRgg&t=1140s&ab_channel=DigitalSreeni), in this manner.

In [73]:
#Calculate density and reconstruction error to find their means values for
density_lst = []

X = dataset[0]
image_in_tensor = X[0]

model_encoder_output_shape = (450,1,1)          # This is the shape of the last layer of the encoder model
out_vector_shape = model_encoder_output_shape[0]*model_encoder_output_shape[1]*model_encoder_output_shape[2]

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    np_converted_encoded_img = Y.detach().numpy()
    flattened = np.reshape(np_converted_encoded_img, (out_vector_shape))
    density = kde.score_samples([flattened])[0]
    density_lst.append(density)
average_density = np.mean(np.array(density_lst))  
stdev_density = np.std(np.array(density_lst)) 
print("The avg of the density values is: ", average_density)
print("The stdev_density of the density values is: ", stdev_density)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

The avg of the density values is:  307.54132174423586
The stdev_density of the density values is:  0.5951443460453201


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Now, it will be shown the density mean and std deviation of the set of anomalies samples

In [75]:
data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the home laptop
# data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the work laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset_anomalies = datasets.ImageFolder(data_anomalies, transform=transform_characteristics)
dataloader_anomalies = torch.utils.data.DataLoader(dataset_anomalies, batch_size=32, shuffle=True)

In [77]:
#Get encoded output of input images = Latent space
encoded_anomalies_images = []

for i in range(len(dataset_anomalies)):
    X = dataset_anomalies[i]
    image_in_tensor = X[0]
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_anomalies_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [78]:
np_encoded_anomaly_images = []
for i in range (len(encoded_anomalies_images)):
    np_conversion = encoded_anomalies_images[i].detach().numpy()
    np_encoded_anomaly_images.append(np_conversion)
np_encoded_anomaly_images = np.array(np_encoded_anomaly_images)
print(type(np_encoded_anomaly_images))

<class 'numpy.ndarray'>


In [79]:
encoded_anomaly_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_anomaly_images]


In [84]:
kde_anomalies = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_anomaly_images_vector)

In [85]:
density_vals_anomalies = kde_anomalies.score_samples(encoded_anomaly_images_vector)
print(density_vals_anomalies)

[308.93296827 308.93296827 309.05065885 308.93296118 309.05065884
 308.93296118]


In [86]:
average_density_anomalies = np.mean(density_vals_anomalies)
stdev_density_anomalies = np.std(density_vals_anomalies)
print("The avg of the density values is: ", average_density_anomalies)
print("The stdev_density of the density values is: ", stdev_density_anomalies)

The avg of the density values is:  308.9721960986903
The stdev_density of the density values is:  0.05548153885019328


Having computed the mean of the kde of the anomaly and non-anomaly images, the following fucntions to decide whether a kde value corresponds or not to an anomaly images can be built.

In [87]:
def ranges_mapper(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [88]:
def map_kde2prob_list(input_list):
    threshold = 307.54          # The mean of the kde corresponding to the non-anomaly images
    std_dev = 0.5951            # The std deviation of the kde corresponding to the non-anomaly images
    prob_score_list = []
    prob_score = 50
    for i in range (len(input_list)):
        aux_score = input_list[i] - threshold
        if aux_score>=0:
            additional = ranges_mapper(aux_score, 0, std_dev, 0, 50)
            prob_score_tot = prob_score+additional
        else:
            additional = ranges_mapper(abs(aux_score), 0, std_dev, 0, 50)
            prob_score_tot = prob_score-additional
        if prob_score_tot > 100:
            prob_score_tot = 100
        if prob_score_tot < 0:
            prob_score_tot = 0
        prob_score_list.append(prob_score_tot)
    return prob_score_list

In [89]:
def map_kde2prob(value):
    threshold = 307.54          # The mean of the kde corresponding to the non-anomaly images
    std_dev = 0.5951            # The std deviation of the kde corresponding to the non-anomaly images
    prob_score = 50
    aux_score = value - threshold
        
    if aux_score>=0:
        additional = ranges_mapper(aux_score, 0, std_dev, 0, 50)
        prob_score_tot = prob_score+additional
    else:
        additional = ranges_mapper(abs(aux_score), 0, std_dev, 0, 50)
        prob_score_tot = prob_score-additional
    if prob_score_tot > 100:
        prob_score_tot = 100
    if prob_score_tot < 0:
        prob_score_tot = 0    
    return prob_score_tot

In [99]:
print(map_kde2prob(307.4))

38.23727104687924


In [102]:
def computePred(kde_value):
    pred = 0
    prob_anomaly = map_kde2prob(kde_value)/100
    if prob_anomaly > 0.5:
        pred = 1
    return pred

In [106]:
print("The prob of the kde value of being anomaly image is: ", map_kde2prob(307.4))
print("Given the probability, it is actually predicted as:", computePred(307.4))

The prob of the kde value of being anomaly image is:  38.23727104687924
Given the probability, it is actually predicted as: 0


Testing the kde thresholds 

In [114]:
test_dir = '../../../Images/BottleStoodUp_atNight/Evaluation'      #This is for the home laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])

dataset_test = datasets.ImageFolder(test_dir, transform=transform_characteristics)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=True)
classes = ('non-anomaly','anomaly')

In [115]:
#Get encoded output of input images = Latent space
encoded_test_imgs = []


for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    np_conversion = Y.detach().numpy()
    encoded_test_imgs.append(np_conversion)
np_encoded_test_images = np.array(encoded_test_imgs)
print(type(np_encoded_test_images))

<class 'numpy.ndarray'>


In [116]:
encoded_test_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_test_images]
print(len(encoded_test_images_vector))

30


In [117]:
density_vals_test = kde.score_samples(encoded_test_images_vector)
print(density_vals_test)

[307.63664979 307.69841459 307.99540734 307.98442734 307.94076871
 307.74247115 307.95797564 307.7468319  307.2463114  307.6606744
 307.39097921 307.51418825 307.30861966 307.58879321 307.86566491
 307.56226119 306.26966807 308.05770585 307.92225246 306.29673812
 306.45407402 305.29079545 305.37480133 305.27951721 306.96603756
 307.65399756 307.65163146 307.80074914 307.97086308 307.65708408]


In [120]:
prob_test = map_kde2prob_list(density_vals_test)

In [123]:
print(prob_test)

[58.12046624837069, 63.30991363414533, 88.26309363139656, 87.34055972501045, 83.67238395654229, 67.01152366179511, 85.11810106764244, 67.3779114900089, 25.324432714520665, 60.139001939887066, 37.47934913280067, 47.831310139078745, 30.559541167279964, 54.09958077561831, 77.36220040013072, 51.8703739659696, 0, 93.49738302647859, 82.11665737161891, 0, 0, 0, 0, 0, 1.7759672438868748, 59.578017144172264, 59.3792183708103, 71.90801078453953, 86.20089769371697, 59.83734533026303]


In [124]:
y_pred = []
y_true = []

# Grabbing only the first image of the anomalies dataset
X = dataset_test[0]
image_in_tensor = X[0]

n_features = len(image_in_tensor[0])  # Get the size of one image of the anomaly images dataset. This is supposed to be 224
for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    ground_truth = X[1]

    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    np_converted_encoded_img = Y.detach().numpy()
    flattened = np.reshape(np_converted_encoded_img, (out_vector_shape))
    density = kde.score_samples([flattened])[0]
    prediction = computePred(density)
    y_pred.append(prediction) # Save Prediction
    y_true.append(ground_truth) # Save Truth
print(y_true)
print(y_pred)

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


Above, it can be seen that all the images are predicted to be anomaly images. This is good for the TPR but it is terrible for the FPR. 